In [5]:
!pip install numexpr text_generation


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import pandas as pd
import datasets
import numexpr  # noqa: F401
import math
from transformers import Tool
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [3]:
class CalculatorTool(Tool):
    name = "calculator"
    description = "This is a tool that calculates. It can be used to perform simple arithmetic operations."

    inputs = ["text"]
    outputs = ["float"]

    def __call__(self, expression):
        local_dict = {"pi": math.pi, "e": math.e}
        output = str(
            numexpr.evaluate(
                expression.strip(),
                global_dict={},  # restrict access to globals
                local_dict=local_dict,  # add common mathematical functions
            )
        )
        return output

In [4]:
def split_answer(row):
    splitted = row["answer"].split("####")
    row["true_reasoning"] = splitted[0]
    row["true_answer"] = float(splitted[1].strip())
    return row


math_dataset = (
    datasets.load_dataset("gsm8k", "main")["train"].shuffle(seed=42).select(range(100))
)
math_dataset = pd.DataFrame(math_dataset)
math_dataset = math_dataset.apply(split_answer, axis=1)
math_dataset = math_dataset.drop(columns=["answer"])
math_dataset = datasets.Dataset.from_pandas(math_dataset)

eval_dataset = math_dataset.select(range(30))

In [5]:
def fake_llm(prompt, stop=None):
    if "special_marker" not in prompt:
        return """
Thought: I should multiply 2 by 3.6452. special_marker
Action:
{
    "action": "calculator",
    "action_input": {"expression": "2*3.6452"}
}
        """
    else:  # We're at step 2
        return """
Thought: I can now answer the initial question
Action:
{
    "action": "final_answer",
    "action_input": {"expression": "3.14159"}
}
            """

In [37]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain.chat_models import ChatHuggingFace

llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
    max_new_tokens=3000,
)
llm_engine = ChatHuggingFace(llm=llm)


def call_llm(input, stop=None):
    return llm_engine.invoke(input, stop=["Observation"]).content

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/aymeric/.cache/huggingface/token
Login successful


In [47]:
from transformers import ReactAgent

agent = ReactAgent(
    llm_engine=call_llm,
    toolbox=[CalculatorTool()],
)

In [50]:
output = agent.run("What is 2 multiplied by 3.6452?")

=====Calling LLM with this prompt:=====
Solve the following task as best you can. You have access to the following tools:

- calculator: This is a tool that calculates. It can be used to perform simple arithmetic operations.
     Takes inputs: ['text']
- final_answer: Provides a final answer to the given problem
     Takes inputs: {'answer': <class 'str'>}


The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (name of the tool to use) and a `action_input` key (input to the tool).

The value in the "action" field should belong to this list: calculator, final_answer.

The $ACTION_JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. Here is an example of a valid $ACTION_JSON_BLOB:

{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}


Make sure to have the $INPUT in the right format for the tool you are using, and do not put variable names as input if you can find the right values.

You will be giv

In [51]:
output

{'answer': '7.2904'}